<a href="https://colab.research.google.com/github/williamweem/one-notebook-ds-project/blob/master/offline-store-recommender-system/Offline_Store_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Package

In [1]:
! pip install surprise

     |████████████████████████████████| 6.5MB 2.5MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675403 sha256=56f43c9169c4237bcd7b235dfe9dc70ce5e272929cb2dc540016b5ab276ecd48
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import pandas as pd
import numpy as np
import time

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import NMF

pd.options.display.float_format = "{:.2f}".format

## Data Upload and Preprocessing

In [0]:
df = pd.read_csv("data untuk olah.csv")

df = df[df["Kategori Pelanggan"]!="UMUM"].copy()
df = df[df["Kode Pelanggan"]!="GRO"].copy()
df = df[df["Kode Pelanggan"]!="HTRSTO"].copy()
df = df[df["DPP+Pajak"]>2].copy()
df = df[~df["Nama Produk"].str.contains("MEMBER CARD")].copy()

le = LabelEncoder()
#
le.fit(df['Kode Pelanggan'])
kode_pelanggan_mapping = dict(zip(le.transform(le.classes_), le.classes_))
df['Kode Pelanggan'] = le.transform(df['Kode Pelanggan']).astype("str")
#
le.fit(df['Kode Produk'])
kode_produk_mapping = dict(zip(le.transform(le.classes_), le.classes_))
df['Kode Produk'] = le.transform(df['Kode Produk']).astype("str")
#
le.fit(df['Kategori Produk'])
kode_kategori_mapping = dict(zip(le.transform(le.classes_), le.classes_))
df['Kategori Produk'] = le.transform(df['Kategori Produk']).astype("str")
#
df["Tanggal"] = pd.to_datetime(df["Tanggal"])
#
df = df.drop("Nama Produk", axis=1)
#
# df_bukti = df.copy()

In [4]:
for i in range(15):
    if i+9>12:
        tahun = "2019"
        if i-3>9:
            bulan = str(i-3)
        else:
            bulan = "0"+str(i-3)
        print("{} {}: {}".format(tahun, bulan, df[df["Tanggal"] < pd.Timestamp(tahun+bulan+"01")]["Kode Pelanggan"].nunique()))
    else:
        tahun = "2018"
        if i+9>9:
            bulan = str(i+9)
        else:
            bulan = "0"+str(i+9)
        print("{} {}: {}".format(tahun, bulan, df[df["Tanggal"] < pd.Timestamp(tahun+bulan+"01")]["Kode Pelanggan"].nunique()))

2018 09: 0
2018 10: 2494
2018 11: 4062
2018 12: 5407
2019 01: 6962
2019 02: 8071
2019 03: 8913
2019 04: 9865
2019 05: 10879
2019 06: 12052
2019 07: 12954
2019 08: 13843
2019 09: 14515
2019 10: 15491
2019 11: 15521


In [5]:
ts = time.time()

set1 = df[df["Tanggal"] <= pd.Timestamp('20181231')].copy()
set2 = df[(df["Tanggal"] >= pd.Timestamp('20190101')) & (df["Tanggal"] <= pd.Timestamp('20190129'))].copy()
set3 = df[(df["Tanggal"] >= pd.Timestamp('20190130')) & (df["Tanggal"] <= pd.Timestamp('20190226'))].copy()
set4 = df[(df["Tanggal"] >= pd.Timestamp('20190227')) & (df["Tanggal"] <= pd.Timestamp('20190326'))].copy()
set5 = df[(df["Tanggal"] >= pd.Timestamp('20190327')) & (df["Tanggal"] <= pd.Timestamp('20190423'))].copy()
set6 = df[(df["Tanggal"] >= pd.Timestamp('20190424')) & (df["Tanggal"] <= pd.Timestamp('20190521'))].copy()
set7 = df[(df["Tanggal"] >= pd.Timestamp('20190522')) & (df["Tanggal"] <= pd.Timestamp('20190618'))].copy()
set8 = df[(df["Tanggal"] >= pd.Timestamp('20190619')) & (df["Tanggal"] <= pd.Timestamp('20190716'))].copy()
set9 = df[(df["Tanggal"] >= pd.Timestamp('20190717')) & (df["Tanggal"] <= pd.Timestamp('20190813'))].copy()
set10 = df[(df["Tanggal"] >= pd.Timestamp('20190814')) & (df["Tanggal"] <= pd.Timestamp('20190910'))].copy()
set11 = df[(df["Tanggal"] >= pd.Timestamp('20190911')) & (df["Tanggal"] <= pd.Timestamp('20190915'))].copy()

train2 = set1.append(set2, ignore_index=True)
train3 = train2.append(set3, ignore_index=True)
train4 = train3.append(set4, ignore_index=True)
train5 = train3.append(set5, ignore_index=True)
train6 = train3.append(set6, ignore_index=True)
train7 = train3.append(set7, ignore_index=True)
train8 = train3.append(set8, ignore_index=True)
train9 = train3.append(set9, ignore_index=True)
train10 = train3.append(set10, ignore_index=True)

time.time()-ts

0.34326982498168945

In [0]:
def to_tabular(df):
    df = df.groupby(["Kode Pelanggan","Kategori Produk"])[["Kategori Produk"]].count()
    df = df.rename(columns={"Kategori Produk":"Jumlah Pembelian"}).reset_index()
    df['Kode Pelanggan'] = df['Kode Pelanggan'].astype(np.int64)
    df['Kategori Produk'] = df['Kategori Produk'].astype(np.int64)
    df = df.sort_values(by=["Kode Pelanggan","Kategori Produk"]).reset_index(drop=True)
    # Klo mau versi dummy
    df['Dummy'] = 1
    # Klo mau versi normalized
    df["Max Pembelian"] = df.groupby("Kategori Produk")[["Jumlah Pembelian"]].transform("max")
    df["Jumlah Norm"] = df["Jumlah Pembelian"] / df["Max Pembelian"]
    df = df.drop(["Max Pembelian"], axis=1)
    return df

def to_pm(df):
    # Pilih values = "Jumlah Pembelian", "Dummy", "Jumlah Norm"
    df = df.pivot(index="Kode Pelanggan", columns="Kategori Produk", values="Dummy").fillna(0).reset_index()
    df.columns.name = None
    df.drop("Kode Pelanggan", axis=1, inplace=True)
    df = df.transpose()
    df = df.reindex(range(0,175), fill_value=0)
    return df

def to_ss(df):
    # Pilih values dan value_name = "Jumlah Pembelian", "Dummy", "Jumlah Norm"
    df = df.pivot(index="Kode Pelanggan", columns="Kategori Produk", values="Dummy").fillna(0).reset_index()
    df.columns.name = None
    df = df.melt(id_vars="Kode Pelanggan", var_name="Kategori Produk", value_name="Dummy")
    return df

def persiapan_test_ss(df, train):
    df = to_ss(to_tabular(df))
    df = df[df["Kode Pelanggan"].isin(df["Kode Pelanggan"].unique()
                                            [np.isin(df["Kode Pelanggan"].unique(), 
                                            train["Kode Pelanggan"].unique())])].copy()
    return df

def ss_result(fit, test, N=5, set=1):
    pred = fit.test(test)
    result = pd.DataFrame(pred, columns=['Kode Pelanggan', 'Kategori Produk', 'Realita', 'Prediksi', 'details'])
    result.drop(columns = {'details'}, inplace = True)
    result = result.sort_values(['Kode Pelanggan','Prediksi'], ascending=False).groupby('Kode Pelanggan').head(N)
    result["Set"]=set
    return result

def pred_summary(df, test_all):
    df = df.groupby(["Kode Pelanggan", "Set"], as_index=False)["Realita"].sum()
    df.rename(columns={"Realita":"Jumlah Prediksi Benar"}, inplace=True)
    # Satuin semua test data disini
    df = df.merge(test_all, on=["Kode Pelanggan","Set"])
    df.rename(columns={"Dummy":"Jumlah Jenis Barang"}, inplace=True)
    df["Precision"] = df["Jumlah Prediksi Benar"]/5
    df["Recall"] = df["Jumlah Prediksi Benar"]/df["Jumlah Jenis Barang"]
    df["F1 Score"] = (2*df["Precision"]*df["Recall"])/(df["Recall"]+df["Precision"])
    df["F1 Score"] = df["F1 Score"].fillna(0)
    return df

def cosine_rec(cs_mat, test):
    result_np_1_cs = np.empty(shape=[0, 3])
    for j in test:
        temp = np.vstack((np.repeat(j, 10) , cs_mat[j].argsort()[-11:-1][::-1] , np.sort(cs_mat[j])[-11:-1][::-1])).T
        result_np_1_cs = np.append(result_np_1_cs, temp, axis=0)
        
    result_1_cs = pd.DataFrame(result_np_1_cs, columns=["Origin", "Kategori Produk", "Prediksi"])
    result_1_cs = result_1_cs.drop_duplicates('Kategori Produk').nlargest(5, 'Prediksi')
    # Kalau kodenya udah mateng, 2 hashtag dibawah bisa dibuka (ngeberatin soalnya)
    # result_1_cs["Origin Item Name"] = result_1_cs["Origin Item"].apply(lambda x: kode_kategori_mapping[x])
    # result_1_cs["Recommended Item Name"] = result_1_cs["Recommended Item"].apply(lambda x: kode_kategori_mapping[x])
    return result_1_cs

def persiapan_test_cs(df, train_tab):
    # Pilih yang di drop = "Jumlah Pembelian", "Dummy", "Jumlah Norm"
    df = to_tabular(df).drop(["Jumlah Pembelian","Jumlah Norm"], axis=1)
    df = df[df["Kode Pelanggan"].isin(df["Kode Pelanggan"].unique()
                                     [np.isin(df["Kode Pelanggan"].unique(), 
                                     train_tab["Kode Pelanggan"].unique())])].copy()
    return df

def testing_cs(fit, train_tab, test, set=1):
    result = pd.DataFrame()
    for i in test["Kode Pelanggan"].unique():
        temp = train_tab[train_tab["Kode Pelanggan"]==i]["Kategori Produk"].unique()
        result_cs = cosine_rec(fit, temp)
        result_cs["Realita"] = 1 * result_cs["Kategori Produk"].isin(test[test["Kode Pelanggan"]==i]["Kategori Produk"].unique())
        result_cs["Kode Pelanggan"] = i
        result = pd.concat([result,result_cs], ignore_index=True)
        print(i)
    result["Set"] = set
    print("Next one")    
    return result

In [7]:
ts = time.time()

train1_ss = to_ss(to_tabular(set1))
train2_ss = to_ss(to_tabular(train2))
train3_ss = to_ss(to_tabular(train3))
train4_ss = to_ss(to_tabular(train4))
train5_ss = to_ss(to_tabular(train5))
train6_ss = to_ss(to_tabular(train6))
train7_ss = to_ss(to_tabular(train7))
train8_ss = to_ss(to_tabular(train8))
train9_ss = to_ss(to_tabular(train9))
train10_ss = to_ss(to_tabular(train10))

test1_ss = persiapan_test_ss(set2, train=train1_ss)
test2_ss = persiapan_test_ss(set3, train=train2_ss)
test3_ss = persiapan_test_ss(set4, train=train3_ss)
test4_ss = persiapan_test_ss(set5, train=train4_ss)
test5_ss = persiapan_test_ss(set6, train=train5_ss)
test6_ss = persiapan_test_ss(set7, train=train6_ss)
test7_ss = persiapan_test_ss(set8, train=train7_ss)
test8_ss = persiapan_test_ss(set9, train=train8_ss)
test9_ss = persiapan_test_ss(set10, train=train9_ss)
test10_ss = persiapan_test_ss(set11, train=train10_ss)

# Disesuaikan apabila "Dummy" atau "Jumlah Norm"
test_ss_all = pd.concat([test1_ss.assign(Set = lambda x:1).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test2_ss.assign(Set = lambda x:2).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test3_ss.assign(Set = lambda x:3).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test4_ss.assign(Set = lambda x:4).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test5_ss.assign(Set = lambda x:5).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test6_ss.assign(Set = lambda x:6).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test7_ss.assign(Set = lambda x:7).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test8_ss.assign(Set = lambda x:8).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test9_ss.assign(Set = lambda x:9).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum(),
                            test10_ss.assign(Set = lambda x:10).groupby(["Kode Pelanggan","Set"], as_index=False)[["Dummy"]].sum()],
                            ignore_index=True)

time.time()-ts

3.950901508331299

## Recommender System using Funk's SVD

In [15]:
ts = time.time()

svd = SVD(n_factors=100,n_epochs=100,lr_all=0.005,reg_all=0.02)
# Klo mau pake "Jumlah Pembelian", rating_scalenya mesti disesuaikan
reader = Reader(rating_scale=(0, 1))

train1_svd = Dataset.load_from_df(train1_ss, reader).build_full_trainset()
train2_svd = Dataset.load_from_df(train2_ss, reader).build_full_trainset()
train3_svd = Dataset.load_from_df(train3_ss, reader).build_full_trainset()
train4_svd = Dataset.load_from_df(train4_ss, reader).build_full_trainset()
train5_svd = Dataset.load_from_df(train5_ss, reader).build_full_trainset()
train6_svd = Dataset.load_from_df(train6_ss, reader).build_full_trainset()
train7_svd = Dataset.load_from_df(train7_ss, reader).build_full_trainset()
train8_svd = Dataset.load_from_df(train8_ss, reader).build_full_trainset()
train9_svd = Dataset.load_from_df(train9_ss, reader).build_full_trainset()
train10_svd = Dataset.load_from_df(train10_ss, reader).build_full_trainset()

test1_svd = Dataset.load_from_df(test1_ss, reader).build_full_trainset().build_testset()
test2_svd = Dataset.load_from_df(test2_ss, reader).build_full_trainset().build_testset()
test3_svd = Dataset.load_from_df(test3_ss, reader).build_full_trainset().build_testset()
test4_svd = Dataset.load_from_df(test4_ss, reader).build_full_trainset().build_testset()
test5_svd = Dataset.load_from_df(test5_ss, reader).build_full_trainset().build_testset()
test6_svd = Dataset.load_from_df(test6_ss, reader).build_full_trainset().build_testset()
test7_svd = Dataset.load_from_df(test7_ss, reader).build_full_trainset().build_testset()
test8_svd = Dataset.load_from_df(test8_ss, reader).build_full_trainset().build_testset()
test9_svd = Dataset.load_from_df(test9_ss, reader).build_full_trainset().build_testset()
test10_svd = Dataset.load_from_df(test10_ss, reader).build_full_trainset().build_testset()

print("Fit 1 Dimulai")
svd1_fit = svd.fit(train1_svd)
print("Fit 2 Dimulai")
svd2_fit = svd.fit(train2_svd)
print("Fit 3 Dimulai")
svd3_fit = svd.fit(train3_svd)
print("Fit 4 Dimulai")
svd4_fit = svd.fit(train4_svd)
print("Fit 5 Dimulai")
svd5_fit = svd.fit(train5_svd)
print("Fit 6 Dimulai")
svd6_fit = svd.fit(train6_svd)
print("Fit 7 Dimulai")
svd7_fit = svd.fit(train7_svd)
print("Fit 8 Dimulai")
svd8_fit = svd.fit(train8_svd)
print("Fit 9 Dimulai")
svd9_fit = svd.fit(train9_svd)
print("Fit 10 Dimulai")
svd10_fit = svd.fit(train10_svd)

print("Tes Dimulai")
pred1_svd = ss_result(svd1_fit,test1_svd, set=1)
print("Tahap 1 Selesai")
pred2_svd = ss_result(svd2_fit,test2_svd, set=2)
print("Tahap 2 Selesai")
pred3_svd = ss_result(svd3_fit,test3_svd, set=3)
print("Tahap 3 Selesai")
pred4_svd = ss_result(svd4_fit,test4_svd, set=4)
print("Tahap 4 Selesai")
pred5_svd = ss_result(svd5_fit,test5_svd, set=5)
print("Tahap 5 Selesai")
pred6_svd = ss_result(svd6_fit,test6_svd, set=6)
print("Tahap 6 Selesai")
pred7_svd = ss_result(svd7_fit,test7_svd, set=7)
print("Tahap 7 Selesai")
pred8_svd = ss_result(svd8_fit,test8_svd, set=8)
print("Tahap 8 Selesai")
pred9_svd = ss_result(svd9_fit,test9_svd, set=9)
print("Tahap 9 Selesai")
pred10_svd = ss_result(svd10_fit,test10_svd, set=10)
print("Tahap 10 Selesai")

pred_svd = pd.concat([pred1_svd, pred2_svd, pred3_svd, pred4_svd, pred5_svd, pred6_svd, pred7_svd, pred8_svd, pred9_svd, pred10_svd], ignore_index=True)
pred_svd_summary = pred_summary(pred_svd, test_all=test_ss_all)

print("Precision value = {}".format(pred_svd_summary["Precision"].mean()))
print("Recall value = {}".format(pred_svd_summary["Recall"].mean()))
print("F1 Score value = {}".format(pred_svd_summary["F1 Score"].mean()))

print("Time Elapsed: {}".format(time.time()-ts))

Fit 1 Dimulai
Fit 2 Dimulai
Fit 3 Dimulai
Fit 4 Dimulai
Fit 5 Dimulai
Fit 6 Dimulai
Fit 7 Dimulai
Fit 8 Dimulai
Fit 9 Dimulai
Fit 10 Dimulai
Tes Dimulai
Tahap 1 Selesai
Tahap 2 Selesai
Tahap 3 Selesai
Tahap 4 Selesai
Tahap 5 Selesai
Tahap 6 Selesai
Tahap 7 Selesai
Tahap 8 Selesai
Tahap 9 Selesai
Tahap 10 Selesai
Precision value = 0.2283593002981077
Recall value = 0.18718915601567285
F1 Score value = 0.17050707428476736
Time Elapsed: 4713.6064121723175


## Recommender System using Non-Negative Matrix Factorization

In [9]:
ts = time.time()

nmf = NMF()
# Klo mau pake "Jumlah Pembelian", rating_scalenya mesti disesuaikan
reader = Reader(rating_scale=(0, 1))

train1_nmf = Dataset.load_from_df(train1_ss, reader).build_full_trainset()
train2_nmf = Dataset.load_from_df(train2_ss, reader).build_full_trainset()
train3_nmf = Dataset.load_from_df(train3_ss, reader).build_full_trainset()
train4_nmf = Dataset.load_from_df(train4_ss, reader).build_full_trainset()
train5_nmf = Dataset.load_from_df(train5_ss, reader).build_full_trainset()
train6_nmf = Dataset.load_from_df(train6_ss, reader).build_full_trainset()
train7_nmf = Dataset.load_from_df(train7_ss, reader).build_full_trainset()
train8_nmf = Dataset.load_from_df(train8_ss, reader).build_full_trainset()
train9_nmf = Dataset.load_from_df(train9_ss, reader).build_full_trainset()
train10_nmf = Dataset.load_from_df(train10_ss, reader).build_full_trainset()

test1_nmf = Dataset.load_from_df(test1_ss, reader).build_full_trainset().build_testset()
test2_nmf = Dataset.load_from_df(test2_ss, reader).build_full_trainset().build_testset()
test3_nmf = Dataset.load_from_df(test3_ss, reader).build_full_trainset().build_testset()
test4_nmf = Dataset.load_from_df(test4_ss, reader).build_full_trainset().build_testset()
test5_nmf = Dataset.load_from_df(test5_ss, reader).build_full_trainset().build_testset()
test6_nmf = Dataset.load_from_df(test6_ss, reader).build_full_trainset().build_testset()
test7_nmf = Dataset.load_from_df(test7_ss, reader).build_full_trainset().build_testset()
test8_nmf = Dataset.load_from_df(test8_ss, reader).build_full_trainset().build_testset()
test9_nmf = Dataset.load_from_df(test9_ss, reader).build_full_trainset().build_testset()
test10_nmf = Dataset.load_from_df(test10_ss, reader).build_full_trainset().build_testset()

print("Fit 1 Dimulai")
nmf1_fit = nmf.fit(train1_nmf)
print("Fit 2 Dimulai")
nmf2_fit = nmf.fit(train2_nmf)
print("Fit 3 Dimulai")
nmf3_fit = nmf.fit(train3_nmf)
print("Fit 4 Dimulai")
nmf4_fit = nmf.fit(train4_nmf)
print("Fit 5 Dimulai")
nmf5_fit = nmf.fit(train5_nmf)
print("Fit 6 Dimulai")
nmf6_fit = nmf.fit(train6_nmf)
print("Fit 7 Dimulai")
nmf7_fit = nmf.fit(train7_nmf)
print("Fit 8 Dimulai")
nmf8_fit = nmf.fit(train8_nmf)
print("Fit 9 Dimulai")
nmf9_fit = nmf.fit(train9_nmf)
print("Fit 10 Dimulai")
nmf10_fit = nmf.fit(train10_nmf)

print("Tes Dimulai")
pred1_nmf = ss_result(nmf1_fit,test1_nmf, set=1)
print("Tahap 1 Selesai")
pred2_nmf = ss_result(nmf2_fit,test2_nmf, set=2)
print("Tahap 2 Selesai")
pred3_nmf = ss_result(nmf3_fit,test3_nmf, set=3)
print("Tahap 3 Selesai")
pred4_nmf = ss_result(nmf4_fit,test4_nmf, set=4)
print("Tahap 4 Selesai")
pred5_nmf = ss_result(nmf5_fit,test5_nmf, set=5)
print("Tahap 5 Selesai")
pred6_nmf = ss_result(nmf6_fit,test6_nmf, set=6)
print("Tahap 6 Selesai")
pred7_nmf = ss_result(nmf7_fit,test7_nmf, set=7)
print("Tahap 7 Selesai")
pred8_nmf = ss_result(nmf8_fit,test8_nmf, set=8)
print("Tahap 8 Selesai")
pred9_nmf = ss_result(nmf9_fit,test9_nmf, set=9)
print("Tahap 9 Selesai")
pred10_nmf = ss_result(nmf10_fit,test10_nmf, set=10)
print("Tahap 10 Selesai")

pred_nmf = pd.concat([pred1_nmf, pred2_nmf, pred3_nmf, pred4_nmf, pred5_nmf, pred6_nmf, pred7_nmf, pred8_nmf, pred9_nmf, pred10_nmf], ignore_index=True)
pred_nmf_summary = pred_summary(pred_nmf, test_all=test_ss_all)

print("Precision value = {}".format(pred_nmf_summary["Precision"].mean()))
print("Recall value = {}".format(pred_nmf_summary["Recall"].mean()))
print("F1 Score value = {}".format(pred_nmf_summary["F1 Score"].mean()))

print("Time Elapsed: {}".format(time.time()-ts))

Fit 1 Dimulai
Fit 2 Dimulai
Fit 3 Dimulai
Fit 4 Dimulai
Fit 5 Dimulai
Fit 6 Dimulai
Fit 7 Dimulai
Fit 8 Dimulai
Fit 9 Dimulai
Fit 10 Dimulai
Tes Dimulai
Tahap 1 Selesai
Tahap 2 Selesai
Tahap 3 Selesai
Tahap 4 Selesai
Tahap 5 Selesai
Tahap 6 Selesai
Tahap 7 Selesai
Tahap 8 Selesai
Tahap 9 Selesai
Tahap 10 Selesai
Precision value = 0.1950953371955694
Recall value = 0.1531902026794957
F1 Score value = 0.141507123988583
Time Elapsed: 1062.348631620407


## Example of Recommendation Given to User (Funk's SVD)

In [17]:
pd.set_option('max_rows', 1000)
pred_svd

,Kode Pelanggan,Kategori Produk,Realita,Prediksi,Set
0,15518,122,1.00,0.97,1
1,15518,83,1.00,0.95,1
2,15518,41,0.00,0.92,1
3,15518,171,0.00,0.92,1
4,15518,172,0.00,0.85,1
...,...,...,...,...,...
88890,125,122,0.00,0.20,10
88891,125,70,0.00,0.19,10
88892,125,172,1.00,0.17,10
88893,125,33,1.00,0.16,10
